In [1]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
# import tflearn

In [2]:
# import pyaudio # source ~./bash... unset PYTHONPATH
# import wave
#
# import librosa
# import librosa.display
# import librosa.feature

# do_rec = input("Test by recording sound [y/n]: ")

# FORMAT = pyaudio.paInt16
# CHANNELS = 1
# RATE = 8000
# CHUNK = 1024
# RECORD_SECONDS = 5
# WAVE_OUTPUT_FILENAME = "test_audio.wav"
# IMG_EXT = ".png"

# if do_rec.lower() == 'y':
#     audio = pyaudio.PyAudio()

#     # start Recording
#     stream = audio.open(format=FORMAT, channels=CHANNELS,
#                     rate=RATE, input=True,
#                     frames_per_buffer=CHUNK)
#     print ("recording...")
#     frames = []

#     for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
#         data = stream.read(CHUNK)
#         frames.append(data)
#     print ("finished recording")


#     # stop Recording
#     stream.stop_stream()
#     stream.close()
#     audio.terminate()

#     waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
#     waveFile.setnchannels(CHANNELS)
#     waveFile.setsampwidth(audio.get_sample_size(FORMAT))
#     waveFile.setframerate(RATE)
#     waveFile.writeframes(b''.join(frames))
#     waveFile.close()
    
# #     plt.figure(figsize=(12, 12))
#     y, sr = librosa.load(WAVE_OUTPUT_FILENAME)
#     S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=256, fmax=4000)
#     librosa.display.specshow(librosa.logamplitude(S,ref_power=np.max),
#                             fmax=4000)
#     output_filename = WAVE_OUTPUT_FILENAME[:-4] + "_spec" + IMG_EXT
#     plt.savefig(output_filename, bbox_inches='tight', pad_inches = 0)
#     librosa.display.specshow(librosa.logamplitude(S,ref_power=np.max),
#                              y_axis='mel', fmax=4000, x_axis='time')
#     plt.colorbar(format='%+2.0f dB')
#     plt.title('Mel spectrogram')
#     plt.tight_layout()

Test by recording sound [y/n]: n


In [3]:
'''
# Convert audio files to spectrograms:



directory = "../test_audio/"

plt.figure(figsize=(12, 12))
for filename in os.listdir(directory):

    if filename[:1] != '.':
        path = os.path.join(directory, filename)

        y, sr = librosa.load(path)
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=256, fmax=4000)
        librosa.display.specshow(librosa.logamplitude(S,ref_power=np.max),
                                fmax=4000)
        output_filename = "../test_spectrograms/" + filename[:-4] + "_spec" + IMG_EXT
        plt.savefig(output_filename, bbox_inches='tight', pad_inches = 0)
'''

'\n# Convert audio files to spectrograms:\n\n\n\ndirectory = "../test_audio/"\n\nplt.figure(figsize=(12, 12))\nfor filename in os.listdir(directory):\n\n    if filename[:1] != \'.\':\n        path = os.path.join(directory, filename)\n\n        y, sr = librosa.load(path)\n        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=256, fmax=4000)\n        librosa.display.specshow(librosa.logamplitude(S,ref_power=np.max),\n                                fmax=4000)\n        output_filename = "../test_spectrograms/" + filename[:-4] + "_spec" + IMG_EXT\n        plt.savefig(output_filename, bbox_inches=\'tight\', pad_inches = 0)\n'

In [3]:
from PIL import Image
import numpy as np
import scipy.ndimage

import PIL




#Returns numpy image at size imageSize*imageSize
def getProcessedData(img,imageSize, grayscale):
    numchannels = 4
    if grayscale:
        numchannels = 2

        
                
    img = img.resize((imageSize,imageSize), resample=Image.ANTIALIAS)
#     print(np.shape(img))
#     print(128*128*4)
# #     imgData = np.asarray(img, dtype=np.uint8).reshape(imageSize,imageSize,1)
#     imgData = np.asarray(img, dtype=np.uint8)
#     print(np.shape(imgData))
    imgData = np.array(img, dtype=np.uint8).reshape(img.size[0], img.size[1], numchannels)
#     print(imgData)

    imgData = imgData[:, :, :-1]
    imgData = imgData/255.
#     print(np.shape(imgData))
#     print(imgData)
#     img.save('greyscale.png')
    return imgData

#Returns numpy image at size imageSize*imageSize
def getImageData(filename,imageSize, grayscale):
    if grayscale:
        img = Image.open(filename).convert('LA')
    else:
        img = Image.open(filename)

#     height, width, channels = scipy.ndimage.imread(filename).shape
#     print("&&")
#     print(height, width, channels)
#     print("&&")
    imgData = getProcessedData(img, imageSize, grayscale)
    return imgData

In [4]:
# from random import shuffle
# import pickle

# directory = "../test_spectrograms/"

# def createDataset(size, validationRatio, testRatio, grayscale):
#     data = []
#     numchannels = 3
#     if grayscale:
#         numchannels = 1
#     count = 1
#     #Add data (X,y)
#     for filename in os.listdir(directory):
#         if filename[:1] != '.':
# #             if count > 10:
# #                 break
#             path = os.path.join(directory, filename)
#             imgData = getImageData(path, size, grayscale)
#             label = 1
#             if filename[:4] == "nois" or filename[:4] == "audi":
#                 label = 0
#             data.append((imgData,label))
# #             count += 1
#     print("data size", np.shape(data))
# #     print(data)
#     #Shuffle data
#     shuffle(data)
# #     print(data)
#     print("data size", np.shape(data))
#     #Extract X and y
#     X,y = zip(*data)
#     print("X size", np.shape(X))
#     print("y size", np.shape(y))
#     #Split data
#     validationNb = int(len(X)*validationRatio)
#     testNb = int(len(X)*testRatio)
#     trainNb = len(X)-(validationNb + testNb)
#     print(validationNb, testNb, trainNb)

#     #Prepare for Tflearn at the same time
#     train_X = np.array(X[:trainNb]).reshape([-1, size, size, 1])
#     train_y = np.array(y[:trainNb])
#     validation_X = np.array(X[trainNb:trainNb+validationNb]).reshape([-1, size, size, 1])
#     validation_y = np.array(y[trainNb:trainNb+validationNb])
#     test_X = np.array(X[-testNb:]).reshape([-1, size, size, 1])
#     test_y = np.array(y[-testNb:])
#     print("    Dataset created!")
        
#     #Save
# #     saveDataset(train_X, train_y, validation_X, validation_y, test_X, test_y, nbPerGenre, genres, sliceSize)

#     return train_X, train_y, validation_X, validation_y, test_X, test_y

# #Saves dataset
# def saveDataset(train_X, train_y, validation_X, validation_y, test_X, test_y):
# #      #Create path for dataset if not existing
# #     if not os.path.exists(os.path.dirname(datasetPath)):
# #         try:
# #             os.makedirs(os.path.dirname(datasetPath))
# #         except OSError as exc: # Guard against race condition
# #             if exc.errno != errno.EEXIST:
# #                 raise

#     #SaveDataset
#     print("[+] Saving dataset... ")
#     datasetName = "main"
#     pickle.dump(train_X, open("train_X_{}.p".format(datasetName), "wb" ))
#     pickle.dump(train_y, open("train_y_{}.p".format(datasetName), "wb" ))
#     pickle.dump(validation_X, open("validation_X_{}.p".format(datasetName), "wb" ))
#     pickle.dump(validation_y, open("validation_y_{}.p".format(datasetName), "wb" ))
#     pickle.dump(test_X, open("test_X_{}.p".format(datasetName), "wb" ))
#     pickle.dump(test_y, open("test_y_{}.p".format(datasetName), "wb" ))
#     print("    Dataset saved! ✅💾")
    



In [5]:
# train_X, train_y, validation_X, validation_y, test_X, test_y = createDataset(128, 0.3, 0.1, True)
# print(np.shape(train_X), np.shape(train_y), np.shape(validation_X), np.shape(validation_y), np.shape(test_X), np.shape(test_y))
# print(np.unique(train_y))


# # print(train_X)
# # print(train_y)
# # print(validation_X) 
# # print(validation_y) 
# # print(test_X) 
# # print(test_y)
# train_y = np.reshape(train_y, (-1, 1))
# test_y = np.reshape(test_y, (-1, 1))
# validation_y = np.reshape(validation_y, (-1, 1))
# print(np.shape(train_X), np.shape(train_y), np.shape(validation_X), np.shape(validation_y), np.shape(test_X), np.shape(test_y))
# saveDataset(train_X, train_y, validation_X, validation_y, test_X, test_y)

data size (1299, 2)
data size (1299, 2)
X size (1299, 128, 128, 1)
y size (1299,)
389 129 781
    Dataset created!
(781, 128, 128, 1) (781,) (389, 128, 128, 1) (389,) (129, 128, 128, 1) (129,)
[0 1]
(781, 128, 128, 1) (781, 1) (389, 128, 128, 1) (389, 1) (129, 128, 128, 1) (129, 1)
[+] Saving dataset... 
    Dataset saved! ✅💾


In [2]:
import pickle
def loadDataset():
    #Load existing
    
    datasetName = "main"
    print("[+] Loading training and validation datasets... ")
    train_X = pickle.load(open("train_X_{}.p".format(datasetName), "rb" ))
    train_y = pickle.load(open("train_y_{}.p".format(datasetName), "rb" ))
    validation_X = pickle.load(open("validation_X_{}.p".format(datasetName), "rb" ))
    validation_y = pickle.load(open("validation_y_{}.p".format(datasetName), "rb" ))
    print("    Training and validation datasets loaded! ✅")
    print("[+] Loading testing dataset... ")
    test_X = pickle.load(open("test_X_{}.p".format(datasetName), "rb" ))
    test_y = pickle.load(open("test_y_{}.p".format(datasetName), "rb" ))
    print("    Testing dataset loaded! ✅")
    return train_X, train_y, validation_X, validation_y, test_X, test_y

train_X, train_y, validation_X, validation_y, test_X, test_y = loadDataset()

print(np.shape(train_X), np.shape(train_y), np.shape(validation_X), np.shape(validation_y), np.shape(test_X), np.shape(test_y))
print(np.unique(train_y))


[+] Loading training and validation datasets... 
    Training and validation datasets loaded! ✅
[+] Loading testing dataset... 
    Testing dataset loaded! ✅
(781, 128, 128, 1) (781, 1) (389, 128, 128, 1) (389, 1) (129, 128, 128, 1) (129, 1)
[0 1]


In [3]:
'''
model taken from this article:
https://chatbotslife.com/finding-the-genre-of-a-song-with-deep-learning-da8f59a61194
'''

import tflearn # source ~./bash...
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

def createModel(imageSize):
    print("[+] Creating model...")
    convnet = input_data(shape=[None, imageSize, imageSize, 1], name='input')

#     convnet = conv_2d(convnet, 64, 2, activation='elu', weights_init="Xavier")
#     convnet = max_pool_2d(convnet, 2)

    convnet = conv_2d(convnet, 128, 4, activation='elu', weights_init="Xavier")
    convnet = max_pool_2d(convnet, 4)

    convnet = conv_2d(convnet, 256, 2, activation='elu', weights_init="Xavier")
    convnet = max_pool_2d(convnet, 2)

    convnet = conv_2d(convnet, 512, 2, activation='elu', weights_init="Xavier")
    convnet = max_pool_2d(convnet, 2)

    
    convnet = fully_connected(convnet, 128, activation='relu')
    convnet = dropout(convnet, 0.5)

    convnet = fully_connected(convnet, 1, activation='softmax') # 2 classes, 1 output
    convnet = regression(convnet, optimizer='rmsprop', loss='binary_crossentropy')


    model = tflearn.DNN(convnet)
    print("    Model created! ✅")
    return model

In [4]:
nbClasses = 2
sliceSize = 128
#Model parameters
batchSize = 128
learningRate = 0.001
nbEpoch = 20

model = createModel(sliceSize)

[+] Creating model...
    Model created! ✅


In [5]:
#Train the model
print("[+] Training the model...")
model.fit(train_X, train_y, n_epoch=nbEpoch, batch_size=batchSize, shuffle=True, 
          validation_set=(validation_X, validation_y), snapshot_step=100, show_metric=True)
print("    Model trained! ✅")

#Save trained model
print("[+] Saving the weights...")
model.save('musicDNN.tflearn')
print("[+] Weights saved! ✅💾")

Training Step: 86  | total loss: 0.61440 | time: 24.431s
| RMSProp | epoch: 013 | loss: 0.61440 - binary_acc: 0.7183 -- iter: 256/781


KeyboardInterrupt: 

In [9]:

#Load weights
print("[+] Loading weights...")
model.load('musicDNN.tflearn')
print("    Weights loaded! ✅")

testAccuracy = model.evaluate(test_X, test_y)[0]
print("[+] Test accuracy: {} ".format(testAccuracy))



[+] Loading weights...
INFO:tensorflow:Restoring parameters from /Users/sangencre/Desktop/NYU/fall_17/ML/final_project/ML_final_project/src/musicDNN.tflearn
    Weights loaded! ✅
[+] Test accuracy: 0.813953488372093 


NameError: name 'denselayer' is not defined

In [17]:
# test on one of the images

path = "./audioclip-1512874035918-5282_spec.png"
imgData = getImageData(path, 128, True)


X_predict = imgData
# X_predict = np.reshape(X_predict, (1 , 128, 128, 1))
print(np.shape(X_predict))
# print(X_predict)
model.load('musicDNN.tflearn')
print(test_y)
model.predict(test_X)
# testAccuracy = model.evaluate(test_X, test_y)[0]
# print("[+] Test accuracy: {} ".format(testAccuracy))

105 24
(128, 128, 1)
INFO:tensorflow:Restoring parameters from /Users/sangencre/Desktop/NYU/fall_17/ML/final_project/ML_final_project/src/musicDNN.tflearn
[[1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
      

In [ ]:
# unfinished attempt to create the same model in TF

import tensorflow as tf

def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 128, 128, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=128,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[4, 4], strides=4)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=256,
      kernel_size=[2, 2],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
        # Convolutional Layer #2 and Pooling Layer #2
    conv3 = tf.layers.conv2d(
      inputs=pool1,
      filters=512,
      kernel_size=[2, 2],
      padding="same",
      activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)